# Sentiment analysis using neural networks

## Import libraries

In [1]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


## Functionality related to text classification, including model construction, training, evaluation, and prediction.

In [2]:

class TextClassifier:
    def __init__(self, max_words=1000, max_sequence_length=100):
        self.max_words = max_words
        self.max_sequence_length = max_sequence_length
        self.tokenizer = Tokenizer(num_words=self.max_words)
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Embedding(self.max_words, 64, input_length=self.max_sequence_length))
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(Dense(3, activation='softmax'))  # 3 classes: negative, neutral, positive
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

    def preprocess_text(self, texts):
        sequences = self.tokenizer.texts_to_sequences(texts)
        return pad_sequences(sequences, maxlen=self.max_sequence_length)

    def train(self, X_train, y_train, epochs=10, batch_size=32):
        self.tokenizer.fit_on_texts(X_train)
        X_train = self.preprocess_text(X_train)
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

    def evaluate(self, X_test, y_test):
        X_test = self.preprocess_text(X_test)
        loss, accuracy = self.model.evaluate(X_test, y_test)
        print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

    def predict(self, texts):
        texts = self.preprocess_text(texts)
        predictions = np.argmax(self.model.predict(texts), axis=-1)
        return predictions


## Read dataframe

### Note the labels 0: "Negative", 1: "Neutral", 2: "Positive"

In [3]:

df = pd.read_csv("GoEmo.csv", sep = ',', header = None, names = ['text', 'label'])
df.head()


,text,label
0,"Wow, this is absolutely breathtaking! You're i...",2
1,I can't get over how amazing your skills are. ...,2
2,This is the kind of content that makes me love...,2
3,I'm in awe of your creativity. How do you come...,2
4,You've outdone yourself once again. I have so ...,2


In [4]:
#df['label'] = df['label'].astype(int)
print(df['label'].value_counts())

label
2    600
0    550
1    250
Name: count, dtype: int64


## Splitting stratified data 

In [5]:

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.3, stratify=df['label'], random_state=42)


## Encode labels

In [6]:

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)


## Initialize and train the model

In [7]:

classifier = TextClassifier()
classifier.train(X_train, y_train_encoded)




Epoch 1/10


25/25 [==============================] - 1s 12ms/step - loss: 1.0485 - accuracy: 0.4388 - val_loss: 1.0475 - val_accuracy: 0.4235
Epoch 2/10
25/25 [==============================] - 0s 5ms/step - loss: 0.9478 - accuracy: 0.5995 - val_loss: 0.9500 - val_accuracy: 0.5765
Epoch 3/10
25/25 [==============================] - 0s 4ms/step - loss: 0.7553 - accuracy: 0.7385 - val_loss: 0.8108 - val_accuracy: 0.6378
Epoch 4/10
25/25 [==============================] - 0s 4ms/step - loss: 0.5150 - accuracy: 0.8074 - val_loss: 0.6687 - val_accuracy: 0.7347
Epoch 5/10
25/25 [==============================] - 0s 5ms/step - loss: 0.3395 - accuracy: 0.8827 - val_loss: 0.5576 - val_accuracy: 0.7653
Epoch 6/10
25/25 [==============================] - 0s 5ms/step - loss: 0.2058 - accuracy: 0.9503 - val_loss: 0.4734 - val_accuracy: 0.8061
Epoch 7/10
25/25 [==============================] - 0s 4ms/step - loss: 0.1092 - accuracy: 0.9949 - val_loss: 0.4115 - val_accuracy: 0.8520
Epoch 8/10
25/2

## Evaluate the model

In [8]:

classifier.evaluate(X_test, y_test_encoded)


14/14 [==============================] - 0s 2ms/step - loss: 0.4216 - accuracy: 0.8500
Test Loss: 0.4216, Test Accuracy: 0.8500


## Example predictions

In [9]:

label_meanings = {0: "Negative", 1: "Neutral", 2: "Positive"}


In [15]:

new_texts = ["This is an awesome product.", "I'm not satisfied with the service.", "The movie left me wondering with more questions than answers."]
predictions = classifier.predict(new_texts)
print("Predictions:", predictions)


1/1 [==============================] - 0s 30ms/step
Predictions: [2 0 1]


In [20]:

for prediction in predictions:
    meaning = label_meanings[prediction]
    print("Predicted Label:", prediction, "Meaning:", meaning)


Predicted Label: 2 Meaning: Positive
Predicted Label: 0 Meaning: Negative
Predicted Label: 1 Meaning: Neutral


In [18]:

new_texts2 = ["I have confidence that joyous actions have a pleasant outcome.", "I found the service to be deplorable making me reluctant for a revisit.", "Being a mystery, the program left me curious to know more."]
predictions2 = classifier.predict(new_texts2)
print("Predictions:", predictions2)


1/1 [==============================] - 0s 20ms/step
Predictions: [2 0 1]


In [19]:

for prediction in predictions2:
    meaning = label_meanings[prediction]
    print("Predicted Label:", prediction, "Meaning:", meaning)


Predicted Label: 2 Meaning: Positive
Predicted Label: 0 Meaning: Negative
Predicted Label: 1 Meaning: Neutral
